<a href="https://colab.research.google.com/github/bzeng18/AIPI531/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
PROJ_DIR = '/content/drive/MyDrive/Colab Notebooks/SA2C_code/Kaggle'
# change current directory after mounting
%cd $PROJ_DIR
! ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SA2C_code/Kaggle
archive-2	     one_hot_encoded.csv   replay_buffer.py	       SNQN1.py		 test.py
data		     pop.py		   report_SNQN_FeatureVec.txt  SNQN_features.py  utility.py
DQN_NS.py	     preprocess_kaggle.py  SA2C.py		       SNQN.py
NextItNetModules.py  __pycache__	   SASRecModules.py	       split_data.py


In [2]:
! pip install pandas trfl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
#without item features
! python SNQN1.py --model=GRU --epoch=5

2023-11-24 03:41:59.216381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 03:41:59.216434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 03:41:59.216468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 03:41:59.241268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 03:42:01.253320: W tensorflow/c

In [6]:
import pandas as pd
item_features1 = pd.read_csv("./archive-2/item_properties_part1.csv")
item_features2 = pd.read_csv("./archive-2/item_properties_part2.csv")
sorted_events = pd.read_csv("./data/sorted_events.csv")
item_features = pd.concat(
            [item_features1, item_features2],
            ignore_index=True,
        )
item_features = item_features[item_features['property']!="available"]
item_features = item_features[item_features["itemid"].isin(sorted_events["item_id"].unique())]
filtered_df = item_features.groupby(['itemid', 'property']).filter(lambda x: x['value'].nunique() == 1)
df = filtered_df
df['property_value'] = df['property'].astype(str) + "_" + df['value'].astype(str)
top_1000_properties = df['property_value'].value_counts().head(300).index
df2 = df[df['property_value'].isin(top_1000_properties)]
one_hot_encoded = df2.pivot_table(index='itemid',
                                          columns='property_value',
                                          aggfunc=len,
                                          fill_value=0)
one_hot_encoded.columns = one_hot_encoded.columns.get_level_values(1)
all_itemids = sorted_events["item_id"].drop_duplicates()
one_hot_encoded_full = pd.DataFrame({'itemid': all_itemids}).merge(one_hot_encoded,
                                                                  on='itemid',
                                                                  how='left').fillna(0)

one_hot_encoded_full.iloc[:, 1:] = one_hot_encoded_full.iloc[:, 1:].astype(int)
one_hot_encoded_full.set_index('itemid', inplace=True)
one_hot_encoded_full.to_csv('one_hot_encoded.csv', index=False)

<ipython-input-6-156f7c064e98>:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  one_hot_encoded_full.iloc[:, 1:] = one_hot_encoded_full.iloc[:, 1:].astype(int)


In [3]:
#With item features
! python SNQN_features.py --model=GRU --epoch=5

2023-11-24 17:59:00.827490: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 17:59:00.827542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 17:59:00.827576: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 17:59:00.835102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 17:59:01.945894: W tensorflow/compiler/